In [13]:
import re
from functools import reduce
import pandas as pd

#global
INPUT_LOCATION = './inputs/day7.txt'
SEPARATOR = '\n'

In [14]:
clean_data = list(map(lambda x: x.rstrip() if len(x) > 1 else x,open(INPUT_LOCATION, 'r').readlines()))  

In [15]:
# for test
#clean_data=clean_data[0:100]
#clean_data

In [16]:
class Bag:
    
    def __init__(self, bag_color):
        self.color = bag_color
        self.content = []
        self.discovered = False

    def __eq__(self, other):
        if isinstance(other, Bag):
            return self.color == other.color
        return False
    
    def add(self, bag):
        self.content.append(bag)
    
    def __repr__(self):
        return 'bag: {!r}\tcontains: {!r}'.format(self.color,self.content) 
    
    def __str__(self):
        return 'bag: {!r}\tcontains: {!r}'.format(self.color,self.content) 

    def __len__(self):
        return len(self.content)

In [106]:
class BagsGraph:
    def __init__(self, bags = []):
        self.bags = bags
        self.dep = []

    def __repr__(self):
        return 'graph with {} Nodes'.format(len(self.bags))
    
    def __str__(self):
        return 'graph with {} Nodes'.format(len(self.bags))
    
    def findIndex(self, bag):
        return self.bags.index(bag)
    
    def findBag(self, bag_color):
        return next(x for x in self.bags if x.color == bag_color)
    
    def dfs(self, bag):
        
        i = self.findIndex(bag)
        self.bags[i].discovered = True
        print(self.bags[i].color)
        self.dep.append(self.bags[i].color)
        for el in self.bags[i].content:
            child = self.findBag(el)
            if not child.discovered:
                # print('discovered: ', child.color)
                self.dfs(child)
            


    
    def append(self, Bag):
        self.bags.append(Bag)

In [107]:
# cleaning data and building a bag dependecy graph
ignored = ['bags,','bags.','bags','bag','bag,','contain']
rules = []
bags_graph = BagsGraph([])
clean = dict()
for raw_rule in clean_data:
    rule = raw_rule.split(' ')
    to_be_deleted = []
    for el in rule:
        if el in ignored:
            to_be_deleted.append(el)
    for el in to_be_deleted:
        rule.remove(el)
    
    print(rule)
    key = rule[0] +' '+ rule[1]
    clean[rule[0] +' '+ rule[1]] = []
    for i in range(3,len(rule)):
        try:
            if i % 3 == 0:
                clean[rule[0] +' '+ rule[1]].append(rule[i] +' '+ rule[i+1]) 
        except:
            clean[rule[0] +' '+ rule[1]] = []
    bag_node = Bag(key)
    bag_node.content = clean[key]
    bags_graph.append(bag_node)

['dark', 'olive', '2', 'muted', 'brown', '1', 'mirrored', 'tomato', '4', 'bright', 'black']
['faded', 'coral', '3', 'drab', 'cyan', '1', 'light', 'aqua', 'bag.']
['plaid', 'plum', '2', 'mirrored', 'cyan']
['clear', 'maroon', '1', 'dotted', 'turquoise', '3', 'dim', 'lavender']
['plaid', 'coral', '3', 'posh', 'fuchsia', '3', 'dotted', 'beige', '2', 'wavy', 'turquoise']
['mirrored', 'indigo', '5', 'pale', 'orange', '5', 'striped', 'aqua', '1', 'dotted', 'cyan', '4', 'muted', 'coral']
['striped', 'brown', '4', 'faded', 'green', '5', 'vibrant', 'red', '3', 'drab', 'black', '3', 'dark', 'orange']
['drab', 'fuchsia', '5', 'shiny', 'chartreuse', '1', 'dark', 'orange', '4', 'drab', 'blue', '1', 'bright', 'chartreuse', 'bag.']
['light', 'fuchsia', '3', 'bright', 'gold', '5', 'clear', 'indigo']
['plaid', 'olive', '4', 'striped', 'indigo', '4', 'muted', 'olive', '2', 'light', 'gray', '2', 'wavy', 'purple']
['dotted', 'green', '2', 'faded', 'yellow']
['dotted', 'cyan', '1', 'light', 'red', '5', 'da

In [108]:
print(bags_graph)

graph with 594 Nodes


In [109]:
bags_graph.findBag('shiny gold')

bag: 'shiny gold'	contains: ['dark salmon', 'wavy purple', 'dark cyan', 'dull chartreuse']

In [110]:
bags_graph.dfs(Bag('shiny gold'))

shiny gold
dark salmon
pale turquoise
vibrant red
faded tan
muted plum
dull red
dotted maroon
posh salmon
dotted chartreuse
dim yellow
bright chartreuse
muted gold
mirrored chartreuse
wavy purple
dark cyan
dull chartreuse
mirrored black
dotted salmon
striped salmon
shiny red


In [112]:
len(bags_graph.dep)

21